In [1]:
##################################################  IMPORTS
import pandas as pd
import pprint
import os
import numpy as np
import time
import math

import sys
#from tabulate import tabulate

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [31]:
##################################################  AUX FUNCTIONS:
from operator import itemgetter as it
#Does the same as unpivot_from_resultdf_to_dict, get_topn and pivot_back_with_topn TOGHETER
#but Im keeping the functions above in case I need to include more logic that I cant on this one
def get_top5_on_lis_of_labels(dfResult, submission):
    lis_of_labels = []
    for _, row in dfResult.iterrows():
        if submission:
            for option in (map(it(0), row[1:].order(ascending=0)[:5].iteritems())):
                lis_of_labels.append([row['id'], option])
        else:        
            lis_of_labels.append(map(it(0), row[1:].order(ascending=0)[:5].iteritems()))
    return lis_of_labels


def get_top5_on_lis_of_labels2F(dfResult, submission):
    list_of_labels = []    

    for _, row in dfResult.iterrows():
        l = []
        done = 0
        pred_country = 'error'
        for option in (
                        map (
                            it(0,1), row[1:].order(ascending=0)[:5].iteritems()
                            )
                      ):
            country = option[0]
            prob = option[1]
            if done == 0:
                pred_country = country
                #if prob >= 0.9: 
                #    pred_country = country
                #    done = 1
                #if prob <= 0.3:                 
                #    pred_country = 'NDF'
                #    done = 1
            if submission:
                list_of_labels.append([row['id'], pred_country]) #for submissions
            else:
                l.append(pred_country)   #for train CV
        
        if submission == False:
            list_of_labels.append(l)
            
    return list_of_labels    



#Unpivot from a dataframe where each prediction is a column to a dict where each prediciton is a row:
def unpivot_from_resultdf_to_dict(dfResult):
    result = {}
    for key in dfResult:
        if key != 'id':
            for i in xrange(0,len(dfResult['id'])):
                if dfResult['id'][i] not in result:
                    result[dfResult['id'][i]] = []
                result[dfResult['id'][i]] += [(key,dfResult[key][i])]
    return result


def get_topn(result,n):
    #Use this to get the top value for each option
    return [x[0] for x in sorted(result,key=lambda x:-x[1])[0:min(len(result),n)]]


def pivot_back_with_topn(result,n, submission,dfResult):
    #keys = sorted([k for k in result])
    #keys = ([k for k in result])
    auxindex = []
    for i in dfResult['id']:
        auxindex.append(i)
    
    keys = ([k for k in auxindex])
    list_of_labels = []
    for key in keys:
        if submission:
            for option in get_topn(result[key],n):
                #print str(key) + ',' + str(option)
                list_of_labels.append([str(key), str(option)] )
        else:
            list_of_labels.append(get_topn(result[key],n))
    return list_of_labels 




#SCORE 
def dcg_at_k(r, k, method=1):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k=5, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


def score_predictions(preds, truth, n_modes=5):
    """
    preds: pd.DataFrame
      one row for each observation, one column for each prediction.
      Columns are sorted from left to right descending in order of likelihood.
    truth: pd.Series
      one row for each obeservation.
    """
    assert(len(preds)==len(truth))
    r = pd.DataFrame(0, index=preds.index, columns=preds.columns, dtype=np.float64)
    for col in preds.columns:
        r[col] = (preds[col] == truth) * 1.0

    score = pd.Series(r.apply(ndcg_at_k, axis=1, reduce=True), name='score')
    return score

In [32]:
##################################################  FILE LOAD AND PRE-PROCESS
#my_train_plus_session3  contains data for action type and action_detail - only rows with sessions - based on pivot_session.csv
#                        scores  0.854761 \ 0.853916 local and  0.87953 LB      73815 train rows
#my_train_plus_session4  same as 3 but conains rows without sessions            213451 train rows  
file = ['my_train_plus_session9.csv', 'age_gender_bkts.csv', 'countries.csv',  'my_test_plus_session9.csv'] #'sessions.csv',
data = {}
for f in file:
    data[f.replace('.csv','')]=pd.read_csv('C:\\git\\Airbnb\\data\\'+f)

#Get train and test data##########################################################################
train = data['my_train_plus_session9']
test = data['my_test_plus_session9']
train = train.fillna(-1)
test = test.fillna(-1)


#train = train[train['country_destination'] != 'NDF']
#train = train[train['country_destination'] != 'US']
#there are 6065 rows on the train data with first_affiliate_tracked null and only 20 on the test data
#train = train[train['first_affiliate_tracked'] != -1]


#Remove ids and prediction field and keep them separate##########################################
train_ids = train['id']
train = train.drop(['id'],axis=1)

target = train['country_destination']
#train = train.drop(['country_destination'],axis=1)
test['country_destination'] = '-'

test_ids = test['id']
test = test.drop(['id'],axis=1)
#np.unique(train[['age']].values)


#This is the Model2 Evaluation target data frame
dfpivottarget = pd.DataFrame(target)
dfpivottarget['AU'] = (dfpivottarget['country_destination'] =='AU')
dfpivottarget['CA'] = (dfpivottarget['country_destination'] =='CA')
dfpivottarget['DE'] = (dfpivottarget['country_destination'] =='DE')
dfpivottarget['ES'] = (dfpivottarget['country_destination'] =='ES')
dfpivottarget['FR'] = (dfpivottarget['country_destination'] =='FR')
dfpivottarget['GB'] = (dfpivottarget['country_destination'] =='GB')
dfpivottarget['IT'] = (dfpivottarget['country_destination'] =='IT')
dfpivottarget['NDF'] = (dfpivottarget['country_destination'] =='NDF') #initially commented for M2, but needed for 2F
dfpivottarget['NL'] = (dfpivottarget['country_destination'] =='NL')
dfpivottarget['other'] = (dfpivottarget['country_destination'] =='other')
dfpivottarget['PT'] = (dfpivottarget['country_destination'] =='PT')
dfpivottarget['US'] = (dfpivottarget['country_destination'] =='US')

In [33]:
len(train.columns)

397

In [34]:
# to_be_dropped = ['about_us',	'account',	'acculynk_bin_check_failed',	'acculynk_bin_check_success',	'acculynk_load_pin_pad',	'acculynk_pin_pad_error',	'acculynk_pin_pad_inactive',	'acculynk_pin_pad_success',	'acculynk_session_obtained',	'active',	'add_business_address_colorbox',	'add_guest_colorbox',	'add_guests',	'add_note',	'agree_terms_check',	'agree_terms_uncheck',	'airbnb_picks',	'airbrb',	'ajax_check_dates',	'ajax_get_referrals_amt',	'ajax_get_results',	'ajax_google_translate',	'ajax_google_translate_description',	'ajax_google_translate_reviews',	'ajax_image_upload',	'ajax_ldp',	'ajax_lwlb_contact',	'ajax_payout_edit',	'ajax_payout_options_by_country',	'ajax_payout_split_edit',	'ajax_photo_widget',	'ajax_photo_widget_form_iframe',	'ajax_price_and_availability',	'ajax_referral_banner_experiment_type',	'ajax_referral_banner_type',	'ajax_refresh_subtotal',	'ajax_send_message',	'ajax_special_offer_dates_available',	'ajax_statsd',	'ajax_worth',	'apply',	'apply_code',	'apply_coupon_click',	'apply_coupon_click_success',	'apply_coupon_error',	'apply_coupon_error_type',	'apply_reservation',	'approve',	'ask_question',	'at_checkpoint',	'authenticate',	'authorize',	'available',	'badge',	'become_user',	'book',	'booking',	'braintree_client_token',	'business_travel',	'calendar_tab_inner2',	'callback',	'campaigns',	'cancel',	'cancellation_policies',	'cancellation_policy_click',	'change',	'change_availability',	'change_currency',	'change_default_payout',	'change_password',	'check',	'city_count',	'clear_reservation',	'click',	'clickthrough',	'collections',	'complete',	'complete_redirect',	'complete_status',	'concierge',	'confirm_email',	'confirmation',	'connect',	'contact_new',	'countries',	'country_options',	'coupon_code_click',	'coupon_field_focus',	'create',	'create_ach',	'create_airbnb',	'create_multiple',	'create_paypal',	'currencies',	'custom_recommended_destinations',	'dashboard',	'deactivate',	'deactivated',	'deauthorize',	'decision_tree',	'delete',	'department',	'departments',	'desks',	'destroy',	'detect_fb_session',	'disaster_action',	'domains',	'edit',	'edit_verification',	'email_by_key',	'email_itinerary_colorbox',	'email_share',	'email_wishlist',	'endpoint_error',	'envoy_bank_details_redirect',	'envoy_form',	'events',	'facebook_auto_login',	'faq',	'faq_category',	'faq_experiment_ids',	'feed',	'forgot_password',	'founders',	'friend_listing',	'friends',	'friends_new',	'glob',	'google_importer',	'guarantee',	'guest_billing_receipt',	'handle_vanity_url',	'hard_fallback_submit',	'has_profile_pic',	'header_userpic',	'home_safety_landing',	'home_safety_terms',	'hospitality',	'hospitality_standards',	'host_2013',	'host_cancel',	'host_summary',	'hosting_social_proof',	'how_it_works',	'identity',	'image_order',	'impressions',	'index',	'invalid_action',	'issue',	'itinerary',	'jumio',	'jumio_redirect',	'jumio_token',	'kba',	'kba_update',	'languages_multiselect',	'life',	'listing',	'listings',	'load_more',	'locale_from_host',	'localization_settings',	'localized',	'locations',	'login',	'login_modal',	'lookup',	'manage_listing',	'media_resources',	'message',	'mobile_landing_page',	'mobile_oauth_callback',	'multi',	'multi_message_attributes',	'my',	'my_listings',	'my_reservations',	'new',	'new_host',	'new_session',	'notifications',	'nyan',	'office_location',	'onenight',	'open_graph_setting',	'open_hard_fallback_modal',	'other_hosting_reviews',	'other_hosting_reviews_first',	'overview',	'p4_refund_policy_terms',	'p4_terms',	'patch',	'pay',	'payment_instruments',	'payment_methods',	'payoneer_account_redirect',	'payoneer_signup_complete',	'payout_delete',	'payout_preferences',	'payout_update',	'pending',	'pending_tickets',	'personalize',	'phone_number_widget',	'phone_verification',	'phone_verification_call_taking_too_long',	'phone_verification_error',	'phone_verification_modal',	'phone_verification_number_submitted_for_call',	'phone_verification_number_submitted_for_sms',	'phone_verification_number_sucessfully_submitted',	'phone_verification_phone_number_removed',	'phone_verification_success',	'photography',	'photography_update',	'place_worth',	'plaxo_cb',	'popular',	'popular_listing',	'populate_from_facebook',	'populate_help_dropdown',	'position',	'press_content',	'press_news',	'press_release',	'pricing',	'print_confirmation',	'privacy',	'profile_pic',	'push_notification_callback',	'qt2',	'qt_reply_v2',	'qt_with',	'questions',	'rate',	'reactivate',	'read_policy_click',	'receipt',	'recent_reservations',	'recommend',	'recommendation_page',	'recommendations',	'recommended_listings',	'redirect',	'references',	'referrer_status',	'refund_guest_cancellation',	'relationship',	'remove_dashboard_alert',	'rentals',	'report',	'reputation',	'request_new_confirm_email',	'request_photography',	'requested',	'requirements',	'reservation',	'reset_calendar',	'respond',	'rest-of-world',	'revert_to_admin',	'review_page',	'reviews',	'reviews_new',	'salute',	'sandy',	'satisfy',	'search',	'search_results',	'set_default',	'set_minimum_payout_amount',	'set_password',	'set_user',	'settings',	'show',	'show_code',	'show_personalize',	'signature',	'signed_out_modal',	'signup_login',	'signup_modal',	'signup_weibo',	'signup_weibo_referral',	'similar_listings',	'similar_listings_v2',	'sldf',	'slideshow',	'social',	'social-media',	'social_connections',	'south-america',	'southern-europe',	'spoken_languages',	'status',	'stpcv',	'sublets',	'submit_contact',	'support_phone_numbers',	'supported',	'sync',	'tell_a_friend',	'terms',	'terms_and_conditions',	'this_hosting_reviews',	'this_hosting_reviews_3000',	'toggle_archived_thread',	'toggle_availability',	'toggle_starred_thread',	'top_destinations',	'tos_2014',	'tos_confirm',	'track_activity',	'track_page_view',	'transaction_history',	'transaction_history_paginated',	'travel',	'travel_plans_current',	'travel_plans_previous',	'trust',	'unavailabilities',	'united-states',	'unread',	'unsubscribe',	'update',	'update_cached',	'update_country_of_residence',	'update_friends_display',	'update_hide_from_search_engines',	'update_message',	'update_notifications',	'update_reservation_requirements',	'upload',	'uptodate',	'use_mobile_site',	'verify',	'view',	'views',	'views_campaign',	'views_campaign_rules',	'webcam_upload',	'weibo_signup_referral_finish',	'why_host',	'widget',	'wishlists',	'zendesk_login_jwt',	'-unknown-',	'Android App Unknown Phone Tablet',	'Android Phone',	'Blackberry',	'Chromebook',	'Linux Desktop',	'Mac Desktop',	'Opera Phone',	'Tablet',	'Windows Desktop',	'Windows Phone',	'iPad Tablet',	'iPhone',	'iPodtouch',	'secs_elapsed']
# train = train.drop(to_be_dropped,axis=1)
# test = test.drop(to_be_dropped,axis=1)

In [35]:
######################       APPLY LOGIC ON TRAIN AND TESTS 
len_train_before = len(train)
len_test_before = len(test)

trainlen = train.shape[0]
dfall = pd.concat((train, test), axis=0, ignore_index=True)

In [36]:
dfall = dfall.drop(['fa_year','ac_year','ac_day','ac_month','fa_day','fa_month'],axis=1) 
dfall = dfall.drop(['accept_decline'],axis=1) #all values are zeros on train, only one ID on test

In [37]:
#NUMBER OF MESSAGES
dfall['num_messages'] = dfall['empty'] +  dfall['10'] + dfall['11'] + dfall['12'] + dfall['15'] + dfall['maybe_information'] + dfall['multi_message'] + dfall['preapproval'] + dfall['special_offer'] + dfall['message_to_host_focus'] + dfall['message_to_host_change'] #+ dfall['ajax_send_message'] + dfall['multi_message_attributes'] + dfall['message'] + dfall['update_message']
dfall = dfall.drop(['empty','10','11','12', '15','maybe_information','multi_message','preapproval','special_offer','message_to_host_focus', 'message_to_host_change'],axis=1) #'ajax_send_message','multi_message_attributes','message','update_message'

dfall.loc[(dfall.num_messages <= 0), 'num_messages'] = 'NoMessages' 
dfall.loc[(dfall.num_messages >= 69), 'num_messages'] = '69+Messages' #train only goes til 69

In [38]:
#set whoever did more than 1 bookng requestes to 1
dfall.loc[dfall.num_AT_booking_request >=1, 'num_AT_booking_request'] = 1
dfall.loc[dfall.guest_booked_elsewhere >=1, 'guest_booked_elsewhere'] = 1

In [39]:
#AGE MANIPULATION
#rememeber that I set everything <14 and >100 as -1 of the way in
dfall.loc[(dfall.age <=14), 'age_bucket'] = '-1' # 0-4 / 5-9 / 10-14
dfall.loc[(dfall.age >=15) & (dfall.age <= 19), 'age_bucket'] = '15-19'
dfall.loc[(dfall.age >=20) & (dfall.age <= 24), 'age_bucket'] = '20-24'
dfall.loc[(dfall.age >=25) & (dfall.age <= 29), 'age_bucket'] = '25-29'
dfall.loc[(dfall.age >=30) & (dfall.age <= 34), 'age_bucket'] = '30-34'
dfall.loc[(dfall.age >=35) & (dfall.age <= 39), 'age_bucket'] = '35-39'
dfall.loc[(dfall.age >=40) & (dfall.age <= 44), 'age_bucket'] = '40-44'
dfall.loc[(dfall.age >=45) & (dfall.age <= 49), 'age_bucket'] = '45-49'
dfall.loc[(dfall.age >=50) & (dfall.age <= 54), 'age_bucket'] = '50-54'
dfall.loc[(dfall.age >=55) & (dfall.age <= 59), 'age_bucket'] = '55-59'
dfall.loc[(dfall.age >=60) & (dfall.age <= 64), 'age_bucket'] = '60-64'
dfall.loc[(dfall.age >=65) & (dfall.age <= 69), 'age_bucket'] = '65-69'
dfall.loc[(dfall.age >=70) & (dfall.age <= 74), 'age_bucket'] = '70-74'
dfall.loc[(dfall.age >=75) & (dfall.age <= 79), 'age_bucket'] = '75-79'
dfall.loc[(dfall.age >=80) & (dfall.age <= 84), 'age_bucket'] = '80-84'
dfall.loc[(dfall.age >=85) & (dfall.age <= 89), 'age_bucket'] = '85-89'
dfall.loc[(dfall.age >=90) & (dfall.age <= 94), 'age_bucket'] = '90-94'
dfall.loc[(dfall.age >=95) & (dfall.age <= 99), 'age_bucket'] = '95-99'
dfall.loc[(dfall.age >=100), 'age_bucket'] = '100+'
dfall = dfall.drop(['age'],axis=1)

In [40]:
#ACTIONS - no improvement
# l=['actionB1','actionB2','actionB3','actionB4','actionB5','actionB6','actionB7','actionB8','actionB9','actionB10','actionB11','actionB12','actionB13','actionB14','actionB15']
# actionBuckets = pd.cut(dfall['num_actions'], 15, retbins=True, labels = l)
# print 'Number of Actions Buckets:', actionBuckets[1]
# dfall['num_actions_bucket'] = actionBuckets[0]
# dfall = dfall.drop(['num_actions'],axis=1)

# dfall.loc[(dfall.num_actions <=82), 'action_bucket'] = 'lessthan82'
# dfall.loc[(dfall.num_actions >=83) & (dfall.num_actions <= 164), 'action_bucket'] = '83-164'
# dfall.loc[(dfall.num_actions >=165) & (dfall.num_actions <= 246), 'action_bucket'] = '165-246'
# dfall.loc[(dfall.num_actions >=247) & (dfall.num_actions <= 328), 'action_bucket'] = '247-328'
# dfall.loc[(dfall.num_actions >=329) & (dfall.num_actions <= 410), 'action_bucket'] = '329-410'
# dfall.loc[(dfall.num_actions >=411) & (dfall.num_actions <= 492), 'action_bucket'] = '411-492'
# dfall.loc[(dfall.num_actions >=493) & (dfall.num_actions <= 574), 'action_bucket'] = '493-574'
# dfall.loc[(dfall.num_actions >=575) & (dfall.num_actions <= 656), 'action_bucket'] = '575-656'
# dfall.loc[(dfall.num_actions >=657) & (dfall.num_actions <= 738), 'action_bucket'] = '657-738'
# dfall.loc[(dfall.num_actions >=739) & (dfall.num_actions <= 820), 'action_bucket'] = '739-820'
# dfall.loc[(dfall.num_actions >=821) & (dfall.num_actions <= 902), 'action_bucket'] = '821-902'
# dfall.loc[(dfall.num_actions >=903) & (dfall.num_actions <= 984), 'action_bucket'] = '903-984'
# dfall.loc[(dfall.num_actions >=985) & (dfall.num_actions <= 1066), 'action_bucket'] = '985-1066'
# dfall.loc[(dfall.num_actions >=1067), 'action_bucket'] = 'morethan1067'
# dfall = dfall.drop(['num_actions'],axis=1)

In [38]:
cols = ['age_bucket', 'gender', 'signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser']
   
dfall2 = dfall[cols].copy()

class MyEncoder:
    labelencoder={}

    def encode(self, d, col, labelencoder={}):
        if col not in labelencoder:
            labelencoder[col] = {}
        result = []
        for i in d:
            if i not in labelencoder[col]:
                labelencoder[col][i] = len(labelencoder[col])+1
                result.append(labelencoder[col][i])
            else:
                result.append(labelencoder[col][i])
        return result,labelencoder

encodings={}
encoder = MyEncoder()

for col in dfall2.columns:
    if dfall2[col].dtypes == object and col != 'age_bucket':
        print 'Encoding: ',col
        dfall2[col], encodings = encoder.encode(dfall2[col].values,col,encodings)    

cols.remove('age_bucket')        

Encoding:  gender
Encoding:  signup_method
Encoding:  language
Encoding:  affiliate_channel
Encoding:  affiliate_provider
Encoding:  first_affiliate_tracked
Encoding:  signup_app
Encoding:  first_device_type
Encoding:  first_browser


In [39]:
#best features
# import operator
# k_best_features = SelectKBestFeatures(dfall2.ix[:, dfall2.columns != 'age_bucket'],dfall2['age_bucket'], 250, False)
# sorted_x  = sorted(k_best_features.items(), key=operator.itemgetter(1))
# (sorted_x).reverse()

# #print sorted_x
# for a,b in sorted_x:
#      print a,b

# KNN GridSearchCV on the age model
# from sklearn.grid_search import GridSearchCV
# def suggestKNeighborsClassifierModel(train, target):
    
#     neighbor_params = [{'n_neighbors' : range(2,6)
#                            #, 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
#                            , 'weights': ['uniform', 'distance']
#                            , 'metric': ['minkowski', 'euclidean', 'manhattan']
#                        }]
#     clf = GridSearchCV(KNeighborsClassifier(), neighbor_params) # , scoring='f1'
#     best = clf.fit(train, target)
#     print best.best_estimator_
    
# suggestKNeighborsClassifierModel(dfall2.loc[(dfall2.age_bucket != '-1')] [cols], 
#                                  dfall2.loc[(dfall2.age_bucket != '-1')]['age_bucket'])

In [12]:
cols =['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider',
  'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
#cols =['gender', 'signup_method','signup_flow', 'signup_app', 'first_device_type']

In [17]:
# agemodel = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
#            metric_params=None, n_neighbors=5, p=2, weights='uniform')
# agemodel.fit (dfall2.loc[(dfall2.age_bucket != '-1')] [cols], dfall2.loc[(dfall2.age_bucket != '-1')] ['age_bucket'])

# preds = agemodel.predict(dfall2[cols])
# dfall2['pred_age_bucket'] = preds
# print len(dfall2.loc[(dfall2.age_bucket != '-1') & (dfall2.age_bucket == dfall2.pred_age_bucket)] )
# print len(dfall2.loc[(dfall2.age_bucket != '-1') & (dfall2.age_bucket == dfall2.pred_age_bucket)] ) / float(len(dfall2.loc[(dfall2.age_bucket != '-1')]) )

# 16975
# 0.230272528725

16975
0.230272528725


In [41]:
params = {'eta': 0.1,
          'max_depth': 10,
          'subsample': 0.5,
          'colsample_bytree': 0.5,
          'objective': 'multi:softmax',
          'num_class': 18}

le = LabelEncoder()
X= dfall2.loc[(dfall2.age_bucket != '-1')] [cols].values
y = le.fit_transform(dfall2.loc[(dfall2.age_bucket != '-1')] ['age_bucket'])   

xgbmodel = xgb.train(params=params, dtrain=xgb.DMatrix(X, y), num_boost_round=100)                    
pred = xgbmodel.predict(xgb.DMatrix(dfall2[cols].values)) 
pred = np.array(pred, dtype=np.int32)

dfall2['pred_age_bucket'] = le.inverse_transform(pred)
print len(dfall2.loc[(dfall2.age_bucket != '-1') & (dfall2.age_bucket == dfall2.pred_age_bucket)] )
print len(dfall2.loc[(dfall2.age_bucket != '-1') & (dfall2.age_bucket == dfall2.pred_age_bucket)] ) / float(len(dfall2.loc[(dfall2.age_bucket != '-1')]) )

# 21276
# 0.288617279596

21276
0.288617279596


In [42]:
dfall['pred_age_bucket'] = le.inverse_transform(pred)
dfall.loc[(dfall.age_bucket == '-1'), 'age_bucket'] = dfall.pred_age_bucket
dfall = dfall.drop(['pred_age_bucket'],axis=1)
#dfall[['age_bucket', 'pred_age_bucket']]

In [41]:
#Population in K by age bucket based on users age bucket
popInKByAgeBucket=pd.read_csv('C:\\git\\Airbnb\\aux_data\\popInKByAgeBucket.csv')
dfall = pd.merge(dfall, popInKByAgeBucket, on='age_bucket', how='left').fillna(0)

In [20]:
#dfall['translate'] = dfall['ajax_google_translate_reviews'] + dfall['ajax_google_translate_description'] + dfall['ajax_google_translate']
#dfall = dfall.drop(['ajax_google_translate_reviews', 'ajax_google_translate_description', 'ajax_google_translate'],axis=1)

In [42]:
######################       DUMMY THE DATASET's COLUMNS - apply one hot encoding
#need to be on the joined dataset to be sure they end up with the same colums
extra_one_hot = ['signup_flow',  'num_AT_booking_request', 'num_AD_pending', 'ajax_google_translate_reviews',
                 'ajax_google_translate_description', 'ajax_google_translate']
                 #, 'requested']

for col in dfall.columns:    
    if (col != 'country_destination' and (dfall[col].dtypes == object or col in extra_one_hot)):#if (dfall[col].dtypes == object or col in extra_one_hot):  
        print 'Dummying: ',col
        dfall_dummy = pd.get_dummies(dfall[col], prefix='is'+col)
        dfall = dfall.drop([col], axis=1)
        dfall = pd.concat((dfall, dfall_dummy), axis=1)
    else:
        print 'Not touching: ',col

train = dfall[:trainlen]
test = dfall[trainlen:]

test = test.drop(['country_destination'],axis=1)

assert(len_train_before == len(train))
assert(len_test_before == len(test))

Dummying:  gender
Dummying:  signup_method
Dummying:  signup_flow
Dummying:  language
Dummying:  affiliate_channel
Dummying:  affiliate_provider
Dummying:  first_affiliate_tracked
Dummying:  signup_app
Dummying:  first_device_type
Dummying:  first_browser
Dummying:  num_AT_booking_request
Dummying:  num_AD_pending
Not touching:  num_actions
Not touching:  start_from_zero
Not touching:  about_us
Not touching:  account
Not touching:  acculynk_bin_check_failed
Not touching:  acculynk_bin_check_success
Not touching:  acculynk_load_pin_pad
Not touching:  acculynk_pin_pad_error
Not touching:  acculynk_pin_pad_inactive
Not touching:  acculynk_pin_pad_success
Not touching:  acculynk_session_obtained
Not touching:  active
Not touching:  add_business_address_colorbox
Not touching:  add_guest_colorbox
Not touching:  add_guests
Not touching:  add_note
Not touching:  agree_terms_check
Not touching:  agree_terms_uncheck
Not touching:  airbnb_picks
Not touching:  airbrb
Not touching:  ajax_check_date

In [43]:
#After encoding, drop all columns that have unique values outise NDF because they wont add prediction to non-NDF classes
#cannot drop on dfall because they were created on the one-hot-encoding
to_be_dropped = []
for col in train.columns:
    if len(np.unique(train[train['country_destination'] != 'NDF'][col])) ==1:
    #if len(np.unique(train[col])) == 1:
        to_be_dropped.append(col)
        
train = train.drop(to_be_dropped,axis=1)
test = test.drop(to_be_dropped,axis=1)
print 'Dropped', len(to_be_dropped), 'columns: ',to_be_dropped

Dropped 107 columns:  ['acculynk_bin_check_failed', 'acculynk_pin_pad_error', 'acculynk_pin_pad_success', 'add_business_address_colorbox', 'add_guest_colorbox', 'ajax_payout_split_edit', 'book', 'braintree_client_token', 'business_travel', 'change_default_payout', 'city_count', 'confirmation', 'create_airbnb', 'custom_recommended_destinations', 'deactivate', 'deactivated', 'disaster_action', 'email_by_key', 'envoy_form', 'events', 'hard_fallback_submit', 'host_cancel', 'localized', 'my_reservations', 'nyan', 'payout_delete', 'phone_verification', 'press_content', 'questions', 'reactivate', 'recommendation_page', 'refund_guest_cancellation', 'rentals', 'report', 'reset_calendar', 'rest-of-world', 'revert_to_admin', 'sandy', 'satisfy', 'set_default', 'set_minimum_payout_amount', 'sldf', 'south-america', 'southern-europe', 'stpcv', 'support_phone_numbers', 'sync', 'this_hosting_reviews_3000', 'tos_2014', 'track_activity', 'unsubscribe', 'use_mobile_site', 'view', 'views', 'views_campaign_

In [48]:
train['about_us'].value_counts(normalize=True)

0     0.998361
1     0.001152
2     0.000312
3     0.000122
4     0.000027
13    0.000014
5     0.000014
dtype: float64

In [50]:
# ##Need to work on this
# #train.groupby(['about_us']).count()
# #train.groupby(['acculynk_bin_check_success']).count().reset_index()
# to_be_dropped = []
# for col in train.ix[:, train.columns != 'country_destination'].columns:
#     x = train[col].value_counts(normalize=True)[0]
#     #print col, x
#     if x >0.99:
#         to_be_dropped.append(col)
        
# #    print col
# #    print train[col].value_counts(normalize=True)
# print 'Dropped', len(to_be_dropped), 'columns: ',to_be_dropped
# train = train.drop(to_be_dropped,axis=1)
# test = test.drop(to_be_dropped,axis=1)

Dropped 331 columns:  ['about_us', 'acculynk_bin_check_success', 'acculynk_load_pin_pad', 'acculynk_pin_pad_inactive', 'acculynk_session_obtained', 'add_guests', 'add_note', 'agree_terms_uncheck', 'airbnb_picks', 'airbrb', 'ajax_get_results', 'ajax_google_translate', 'ajax_google_translate_description', 'ajax_ldp', 'ajax_payout_edit', 'ajax_payout_options_by_country', 'ajax_photo_widget', 'ajax_price_and_availability', 'ajax_send_message', 'ajax_special_offer_dates_available', 'ajax_worth', 'apply', 'apply_code', 'apply_coupon_click', 'apply_coupon_click_success', 'apply_coupon_error', 'apply_coupon_error_type', 'approve', 'authorize', 'badge', 'become_user', 'booking', 'cancel', 'cancellation_policy_click', 'change', 'change_availability', 'change_currency', 'change_password', 'check', 'clear_reservation', 'clickthrough', 'complete', 'concierge', 'contact_new', 'countries', 'country_options', 'coupon_code_click', 'coupon_field_focus', 'create_ach', 'create_multiple', 'create_paypal', 

In [74]:
len(train['about_us'].value_counts(normalize=True))# [0] 

7

In [12]:
#OR ENCODE THE DATASET - transfors strings into integers
#If one  hot encoding was applied this step wont do anything
##add country_dest filter - not working at the moment
# class MyEncoder:
#     labelencoder={}

#     def encode(self, d, col, labelencoder={}):
#         if col not in labelencoder:
#             labelencoder[col] = {}
#         result = []
#         for i in d:
#             if i not in labelencoder[col]:
#                 labelencoder[col][i] = len(labelencoder[col])+1
#                 result.append(labelencoder[col][i])
#             else:
#                 result.append(labelencoder[col][i])
#         return result,labelencoder

# encodings={}
# encoder = MyEncoder()

# #pprint_df(foo,10)
# for col in train.columns:
#     if train[col].dtypes == object :
#         print 'Encoding: ',col
#         train[col], encodings = encoder.encode(train[col].values,col,encodings)
#         test[col],  encodings = encoder.encode(test[col].values,col,encodings)   

In [71]:
#train[train['country_destination'] != 'NDF'][0:100].to_csv('C:\\git\\Airbnb\\trainNoCSV.csv',index=False) 

In [23]:
######################       SELECT K BEST FEARURES METHOD
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif

def SelectKBestFeatures(train, target, k, debug=False):
    k_best = SelectKBest(f_classif, k='all')# score all features    
    k_best.fit(train, target) 
    scores = k_best.scores_     
    unsorted_pairs = zip(train.columns, scores)# joins the labels with the values 
    sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))

    k_best_features = dict(sorted_pairs[:k])
    if debug:
        print sorted_pairs
        print "{0} best features: {1}\n".format(k, k_best_features.keys())

    return k_best_features
# import operator
# k_best_features
# sorted_x  = sorted(k_best_features.items(), key=operator.itemgetter(1))
# (sorted_x).reverse()
# print sorted_x

#CREATE LIST OF COMBINATIONS BETWEEN FEATURES
#new_list = k_best_features.keys()
#import itertools
#all = []
#for i in range(len(new_list)):
#    if i !=0: #already looked at individual features on step 1. Start by 2X2
#        all.extend([sorted(l) for l in itertools.combinations(new_list, i+1)])

In [24]:
######################SELECT K-BEST FEATURES - filtering out NDFs
#Remove columns that we dont have data on the training side, otherweise it will messup the select best features
#If I want to automate the removal, it has to be here; it cannot be before the df_all split; they wont be removed because there
#are values on the test rows but we cant do anything with them because there is nothing on train

import operator
#k_best_features = SelectKBestFeatures(train[train['country_destination'] != 'NDF'].ix[:, train.columns != 'country_destination'],target[target != 'NDF'], 1000, False)
k_best_features = SelectKBestFeatures(train.ix[:, train.columns != 'country_destination'],target, 100, False)
sorted_x  = sorted(k_best_features.items(), key=operator.itemgetter(1))
(sorted_x).reverse()

#print sorted_x
for a,b in sorted_x:
     print a,b

isnum_AT_booking_request_0 1077.4059948
isnum_AT_booking_request_1 1077.4059948
isage_bucket_-1 989.443022457
PopInKtoUS 984.831965543
PopInKtoGB 984.535632154
PopInKtoCA 983.605339115
PopInKtoAU 981.424585907
PopInKtoFR 978.684418889
PopInKtoNL 952.182235412
PopInKtoDE 943.750706183
PopInKtoPT 942.425798601
PopInKtoIT 908.720259442
PopInKtoES 899.7677045
isnum_AD_pending_0 857.583626757
isnum_AD_pending_1 792.292685375
isnum_messages_69+Messages 652.187063015
requested 626.838889236
verify 519.830254662
isgender_-unknown- 422.063096018
confirm_email 270.359993306
isnum_messages_1.0 246.893848849
travel_plans_current 229.740820024
pending 214.01835416
at_checkpoint 175.623947073
secs_elapsed 160.280057515
isgender_FEMALE 153.716400511
isage_bucket_25-29 147.617298091
issignup_app_Web 147.118309256
isfirst_browser_-unknown- 145.389501685
isage_bucket_30-34 144.51121516
identity 142.065549665
isgender_MALE 135.1437891
issignup_flow_0 134.613271295
dashboard 132.017824524
cancellation_pol

In [52]:
######################       GRIDSEARCHCV to check best models - each one can take hours even days to run
from sklearn.grid_search import GridSearchCV
def suggestRandomForestClassifierModel():
    
    forest_params = [{
                    "criterion": ["gini", "entropy"],
                    "max_features": range(1, 6),            
                    "max_depth": range(2, 6),
                    #"min_samples_split": sp_randint(1, 11),
                    #"min_samples_leaf": sp_randint(1, 11),
                    "bootstrap": [True, False]
                    }]
    clf = GridSearchCV(RandomForestClassifier(n_estimators=50), forest_params) # , scoring='f1'
    best = clf.fit(train, target)
    print best.best_estimator_

    
def suggestKNeighborsClassifierModel():
    
    neighbor_params = [{'n_neighbors' : range(1,10)
                           , 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
                           , 'weights': ['uniform', 'distance']
                           , 'metric': ['minkowski', 'euclidean', 'manhattan']
                           , 'p' : range(1,10)
                       }]
    clf = GridSearchCV(KNeighborsClassifier(), neighbor_params) # , scoring='f1'
    best = clf.fit(train, target)
    print best.best_estimator_
    

def suggestXGBClassifierModel1():
    xgb_params = [{  
                  'learning_rate': [0.25,0.3,0.35]
                 , 'n_estimators' : [20,30,40,45]
                 , 'gamma' : range(0,3)
                 , 'max_depth': range(4,6)
                 , 'min_child_weight' : range (0,4)
                 , 'max_delta_step': range (1,4)
                 , 'subsample': [0.5,1]
                 , 'colsample_bytree': [0.5,1]
                }]
    
    clf = GridSearchCV(XGBClassifier(objective='multi:softprob'), xgb_params) #, scoring='f1'
    best = clf.fit(train[k_best_features.keys()], target)
    print best.best_estimator_


def suggestXGBClassifierModel2F():
    xgb_params = [{                   
                  'gamma' : range(0,10)
                 , 'max_depth': range(3,10)
                 , 'min_child_weight' : range (0,10)
                 , 'max_delta_step': range (1,10)
                 , 'subsample': [0.4,0.5,0.6,1]
                 , 'colsample_bytree': [0.4,0.5,0.6,1]
                }]

    clf = GridSearchCV(XGBClassifier(learning_rate=0.3, n_estimators=25,objective='multi:softprob') 
                       , xgb_params) # , scoring='f1'


    best = clf.fit(train, target)
    print best.best_estimator_


model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)      
    
def suggestXGBClassifierModel2(train,target):
    xgb_params = [{                   
#                    'colsample_bylevel': [0.5, 1]
#                  , 'colsample_bytree':  [0.5, 1]
                   'gamma' : range(1,3)
                 , 'learning_rate' : [0.3, 0.4]
                 , 'max_delta_step': range (1,3)
                 , 'max_depth': range(3,6)
                 , 'min_child_weight' : range (0,2)
                 , 'n_estimators' : [30,40]
                 , 'subsample': [0.5,1]
                }]

    clf = GridSearchCV(XGBClassifier(objective='binary:logistic'), xgb_params) # , scoring='f1'
    
    for col in target.columns:
        if col != 'country_destination': 
            print 'Testing', col
            best = clf.fit(train, target[col])
            
            f = open('C:\\git\\Airbnb\\GridSearch\\'+col+'.txt', 'w')
            f.write(col)
            f.write('\n')
            f.write(str(best.best_estimator_))
            f.close()
   
#footrain =  train.ix[:, train.columns != 'country_destination'][:1000]
#footarget = dfpivottarget[:1000]

#print "Start" + time.strftime("%c")    
#suggestXGBClassifierModel2( train.ix[:, train.columns != 'country_destination'], dfpivottarget)
#print "End" + time.strftime("%c")
#suggestXGBClassifierModel3( train.ix[:, train.columns != 'country_destination'], dfpivottarget)

In [22]:
from sklearn.grid_search import GridSearchCV
class XGBoostClassifierForGRIDSEARCHCV():
    def __init__(self, num_boost_round=10, **params):
        self.clf = None
        self.num_boost_round = num_boost_round
        self.params = params
        self.params.update({'objective': 'binary:logistic'})
 
    def fit(self, X, y, num_boost_round=None):
        num_boost_round = num_boost_round or self.num_boost_round
        self.label2num = dict((label, i) for i, label in enumerate(sorted(set(y))))
        dtrain = xgb.DMatrix(X, label=[self.label2num[label] for label in y])
        self.clf = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=num_boost_round)
 
    def predict(self, X):
        num2label = dict((i, label)for label, i in self.label2num.items())
        Y = self.predict_proba(X)
        y = np.argmax(Y, axis=1)
        return np.array([num2label[i] for i in y])
 
    def predict_proba(self, X):
        dtest = xgb.DMatrix(X)
        return self.clf.predict(dtest)
 
    def score(self, X, y):
        Y = self.predict_proba(X)
        return 1 / logloss(y, Y)
 
    def get_params(self, deep=True):
        return self.params
 
    def set_params(self, **params):
        if 'num_boost_round' in params:
            self.num_boost_round = params.pop('num_boost_round')
        if 'objective' in params:
            del params['objective']
        self.params.update(params)
        return self
    
    
def logloss(y_true, Y_pred):
    label2num = dict((name, i) for i, name in enumerate(sorted(set(y_true))))
    return -1 * sum(math.log(y[label2num[label]]) if y[label2num[label]] > 0 else -np.inf for y, label in zip(Y_pred, y_true)) / len(Y_pred)


clf = XGBoostClassifierForGRIDSEARCHCV(
    eval_metric = 'ndcg',
    num_class = 2,
    num_boost_round = 80,
    max_depth = 12,
    subsample = 0.5,
    colsample_bytree = 1.0,
    silent = 1
    )
parameters = {
    'num_boost_round': [100, 250, 500],
    'eta': [0.1, 0.3],
    'max_depth': [6, 9, 12],
    'subsample': [0.9, 1.0],
    'colsample_bytree': [0.9, 1.0],
}
clf = GridSearchCV(clf, parameters, cv=2)

   
# for col in dfpivottarget.columns:
#     if col != 'country_destination': 
#     #if col == 'US':  
#         f = open('C:\\git\\Airbnb\\GridSearch\\XGB'+col+'.txt', 'w')  
#         f.write(col)
#         f.write('\n')
#         print 'Testing', col
#         clf.fit(train.ix[:, train.columns != 'country_destination'].values , dfpivottarget[col].values)
#         best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
#         print(score)
#         f.write(str(score))
#         f.write('\n')         
#         for param_name in sorted(best_parameters.keys()):
#             print("%s: %r" % (param_name, best_parameters[param_name]))
#             f.write(str("%s: %r" % (param_name, best_parameters[param_name])))
#             f.write('\n')            
#         f.close()
#https://github.com/dmlc/xgboost/blob/master/doc/parameter.md        

In [47]:
best_parameters.keys()

['eta', 'max_depth']

In [44]:
clf.grid_scores_

[mean: 0.95324, std: 0.00320, params: {'eta': 0.1},
 mean: 0.75452, std: 0.00515, params: {'eta': 0.3}]

In [44]:
#SCORE AND CROSS VALIDATE
#This methos applies cross validation on the train set. It is configured to always do on two ways refered as model1 and model2
#mode1 (M1) and model2 (M2F) dont actually refer to the model itself but he way the model is being evaluated. 
#model 2 itself is also computed in 2 ways, full dataset (M2F) and noNDF (just M2)
#on model1: the prediction is in one categorical column; on model2 each possible prediction value corresponds to a dummy colum
#so it fits N model on the same dataset, N being the number of possible categories

from sklearn.cross_validation import StratifiedShuffleSplit
def crossValidate(model1, model2, xgbparams, trainCV, targetCV, nfolds, debug, runM1=False, runM2=False, runM2F=False, runM3=False):
    print "Cross Validating..."
    cv = StratifiedShuffleSplit(targetCV['country_destination'], nfolds, test_size  = 0.3, random_state = 42)

    foldercount=0
    list_scores1 = [] 
    list_scores2 = [] 
    list_scores2F= [] 
    list_scores3= [] 
    avgscore1 = 0
    avgscore2 = 0
    avgscore2F = 0    
    avgscore3 = 0  
    
    for train_index, test_index in cv: 
        if debug:
            print 'Go for fold: ',foldercount    
            #print("TRAIN:", train_index, "TEST:", test_index)  
        train_features = []
        train_labels = []
        test_features = []
        test_labels = []    
                
        train_features, train_labels = trainCV.iloc[train_index], targetCV.iloc[train_index]
        test_features, test_labels =   trainCV.iloc[test_index],  targetCV.iloc[test_index]        
          
        
        #create an array from a <class 'pandas.core.series.Series'> to attch to the final DF - mainly for debugging
        truth = []
        for i in test_labels['country_destination']:
            truth.append(i)
        
        #these are for the model being trained without NDF, so I have to filter the NDF values and the NDF column
        train_featuresNoNDF = train_features[train_features['country_destination'] != 'NDF'].ix[:, train_features.columns != 'country_destination']
        test_featuresNoNDF = test_features.ix[:, test_features.columns != 'country_destination']
               
        #this is for M1 and M2F  so i just remove the NDF column that was passed in for the previous filter
        train_features = train_features.ix[:, train_features.columns != 'country_destination']
        test_features = test_features.ix[:, test_features.columns != 'country_destination']

        
######## MODEL1 #####################################
        if runM1:
            model1.fit(train_featuresNoNDF, train_labels[train_labels['country_destination'] != 'NDF']['country_destination']) #train_labels['country_destination']
            pred_array = model1.predict_proba(test_featuresNoNDF)     


            my_debugger = []
            list_of_labels1 = []        
            for idx, val in enumerate(pred_array):
                mypred = (zip(model1.classes_, pred_array[idx]))            
                sorted_by_pred = sorted(mypred, key=lambda tup: tup[1], reverse=True)
                sorted_by_pred = sorted_by_pred[:5]                      
                only_labels =  [x[0] for x in sorted_by_pred]            
                #apply prob correction?

                list_of_labels1.append(only_labels)
                if debug:
                    my_debugger.append ([x[0] +': '+ str(x[1]) for x in sorted_by_pred])        


            #list_of_labels contains a list where each element is a list with the top5 predictions for that ID, for example:
            #[['NDF', 'US', 'other', 'ES', 'NL'], ['NDF', 'US', 'other', 'FR', 'IT'],]         
            preds_df = pd.DataFrame(list_of_labels1, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores1 = score_predictions(preds_df, truth)        
            avgscore1 = np.average(scores1)
            list_scores1.append(avgscore1)       

        
        #This is used by M2 and M2F:
        #get the ids from the original train ids based on the indexes we are testing:
        ids = []
        for i in train_ids.iloc[test_index]: # train_ids
            ids.append(i)    
            
            
######## MODEL2 #####################################
        if runM2:
            #pred_list contains the country and a list of probabilities the row belong to that country for that country
            #1. Do the prediction for each country
            pred_list = []
            for col in targetCV.columns:
                if col != 'country_destination' and col != 'NDF': #we dont want to predict NDF on this               
                    model2.fit(train_featuresNoNDF, train_labels[train_labels['country_destination'] != 'NDF'][col])
                    pred = model2.predict_proba(test_featuresNoNDF) 
                    pred_list.append([col, pred])


            #2.Create the result data frame
            dfResult = pd.DataFrame()               
            dfResult['id'] = ids


            for a in pred_list: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult[dest] = values                        
            #at this point, dfResult contains one row per ID with one colum for each country and 
            #the True prediction probability for that country. 

            #The following Lines have been deprecated and subistituded by the get_top5_on_lis_of_labels3 function
            #Unpivot: #result = unpivot_from_resultdf_to_dict(dfResult)
            #Get topp5: #list_of_labels = pivot_back_with_topn(result,5, False, dfResult)        
            #return dfResult
            #function "2" applies the "NDF correction" because NDF wasn't one of the predictions
            #return dfResult
            list_of_labels2 = get_top5_on_lis_of_labels2(dfResult, False)

            preds_df = pd.DataFrame(list_of_labels2, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores2 = score_predictions(preds_df, truth)
            avgscore2 = np.average(scores2)
            list_scores2.append(avgscore2)  

######## MODEL2F #####################################
#pretty much the same as the previous model - except fot the function being called 
#TODO: maybe unify the code in one method
        if runM2F:
            pred_listF = []
            for col in targetCV.columns:
                if col != 'country_destination': 
                    model2.fit(train_features, train_labels[col])
                    pred = model2.predict_proba(test_features) 
                    pred_listF.append([col, pred])


            #2.Create the result data frame
            dfResult2F = pd.DataFrame()                
            dfResult2F['id'] = ids# can use the same ids created on the previous step

            for a in pred_listF: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult2F[dest] = values                
        
            list_of_labels2F = get_top5_on_lis_of_labels2F(dfResult2F, False)   


            preds_df2F = pd.DataFrame(list_of_labels2F, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores2F = score_predictions(preds_df2F, truth)
            avgscore2F = np.average(scores2F)
            list_scores2F.append(avgscore2F)          

######## MODEL3 #####################################
        if runM3:
            pred_listM3 = []
            for col in targetCV.columns:
                if col != 'country_destination':
                    xgbmodel = xgb.train(params=xgbparams, dtrain=xgb.DMatrix(train_features.values, train_labels[col].values), num_boost_round=100)                    
                    pred = xgbmodel.predict(xgb.DMatrix(test_features.values)) 
                    pred_listM3.append([col, pred])

            #2.Create the result data frame
            dfResult3 = pd.DataFrame()               
            dfResult3['id'] = ids


            for a in pred_listM3: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult3[dest] = values     
                
            list_of_labels3 = get_top5_on_lis_of_labels2F(dfResult3, False)

            preds_df3 = pd.DataFrame(list_of_labels3, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores3 = score_predictions(preds_df3, truth)
            avgscore3 = np.average(scores3)
            list_scores3.append(avgscore3)              

        
##################
        if debug:
            print 'fold:', foldercount, 'score:', avgscore1,'|',avgscore2,'|',avgscore2F, '|',avgscore3
            ################MODEL1 DEBUG
            if runM1:
                debbuger_df = pd.DataFrame(my_debugger, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))   
                debbuger_df['t']  = truth            
                debbuger_df['score']  = scores1
                debbuger_df['id'] = ids        
                debbuger_df.to_csv('C:\\git\\Airbnb\\LOG_EvalModel1__'+str(foldercount)+'.csv',index=False)   

            ################MODEL2 DEBUG
            if runM2:
                dfResult['t']  = truth            
                dfResult['score']  = scores2
                foo = pd.concat((dfResult, preds_df), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel2__'+str(foldercount)+'.csv',index=False)
            
            ################MODEL2F DEBUG
            if runM2F:
                dfResult2F['t']  = truth            
                dfResult2F['score']  = scores2F
                foo = pd.concat((dfResult2F, preds_df2F), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel2F__'+str(foldercount)+'.csv',index=False)

            ################MODEL3 DEBUG
            if runM3:
                dfResult3['t']  = truth            
                dfResult3['score']  = scores3
                foo = pd.concat((dfResult3, preds_df3), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel3__'+str(foldercount)+'.csv',index=False)                
            
        foldercount +=1

    return np.average(list_scores1), np.average(list_scores2), np.average(list_scores2F), np.average(list_scores3)

# model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
#        min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
#        objective='multi:softprob', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

# model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
#        min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
#        objective='binary:logistic', reg_alpha=0, reg_lambda=1,       
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

# xgbparams = {'eta': 0.2,
#       'max_depth': 6,
#       'subsample': 0.5,
#       'colsample_bytree': 0.5,
#       'objective': 'binary:logistic',
#       'num_class': 2}  

# #features =  train.ix[:, train.columns != 'country_destination'].columns  #k_best_features.keys(
# #features = train.columns
# #train = train[:30000]
# #dfpivottarget = dfpivottarget[:30000]

# mycolumns = ['feature_list', 'scoreM1', 'ScoreM2NoNDF', 'ScoreM2Full', 'ScoreM3']
# resultdf2 = pd.DataFrame(columns=mycolumns)

# features = train [ [col for col in train.columns if col not in ['start_from_zero', 
#                                                                 'isstart_from_zero_0', 'isstart_from_zero_1',
#                                                                 'num_actions', ]] ].columns
 
# score1, ScoreM2NoNDF, ScoreM2Full, Score3 = crossValidate(model1, model2, xgbparams, train[features], dfpivottarget, 
#                                                           nfolds = 1, 
#                                                           debug= True,
#                                                           runM1=False, 
#                                                           runM2=False, 
#                                                           runM2F=True, 
#                                                           runM3=False)

# resultdf2.loc[len(resultdf2)] = (features, score1, ScoreM2NoNDF, ScoreM2Full, Score3)
# resultdf2

In [50]:
######################################################################################################################
#################################################DEBUG################################################################
######################################################################################################################
from sklearn.cross_validation import StratifiedShuffleSplit
trainCV = train
targetCV = dfpivottarget

cv = StratifiedShuffleSplit(targetCV['country_destination'], 1, test_size  = 0.3, random_state = 42)

foldercount=0
list_scores1 = [] 
list_scores2 = [] 
list_scores2F= [] 
avgscore1 = 0
avgscore2 = 0
avgscore2F = 0    
params  = {'eta': 0.1,
          'max_depth': 6,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'objective': 'binary:logistic',
          'num_class': 2
          } 

for train_index, test_index in cv: 
    train_features = []
    train_labels = []
    test_features = []
    test_labels = []    

    train_features, train_labels = trainCV.iloc[train_index], targetCV.iloc[train_index]
    test_features, test_labels =   trainCV.iloc[test_index],  targetCV.iloc[test_index]        

    #create an array from a <class 'pandas.core.series.Series'> to attch to the final DF - mainly for debugging
    truth = []
    for i in test_labels['country_destination']:
        truth.append(i)

    #this is for M1 and M2F  so i just remove the NDF column that was passed in for the previous filter
    train_features = train_features.ix[:, train_features.columns != 'country_destination']
    test_features = test_features.ix[:, test_features.columns != 'country_destination']

    #pred_list contains the country and a list of probabilities the row belong to that country for that country
    #1. Do the prediction for each country
    #model2.fit(train_features, train_labels[col])
    #pred = model2.predict_proba(test_features) 
    #pred_listF.append([col, pred])
        
        
    pred_list = []
    for col in targetCV.columns:
        if col != 'country_destination':
            X = train_features.values            
            y = train_labels[col].values
            
            #print test_features
            clf1 = xgb.train(params=params, dtrain=xgb.DMatrix(X, y), num_boost_round=100)                    
            pred = clf1.predict(xgb.DMatrix(test_features.values)) 
            pred_list.append([col, pred])

    ids = []
    for i in train_ids.iloc[test_index]: # train_ids
        ids.append(i)
        
    #2.Create the result data frame
    dfResult = pd.DataFrame()               
    dfResult['id'] = ids


    for a in pred_list: # gets the predictions, one country at a time
        dest = a[0] #get country name"
        predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
        values = []
        for p in predictions:
            values.append(p[1])
        dfResult[dest] = values                        

    list_of_labels2 = get_top5_on_lis_of_labels2F(dfResult, False)

    preds_df = pd.DataFrame(list_of_labels2, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
    scores2 = score_predictions(preds_df, truth)
    avgscore2 = np.average(scores2)
    list_scores2.append(avgscore2)  

In [77]:
pred_list[7][1]

array([[ 0.60919112,  0.61028111],
       [ 0.59411204,  0.62901479],
       [ 0.60409683,  0.60948884],
       ..., 
       [ 0.58081907,  0.61655861],
       [ 0.63605601,  0.47351736],
       [ 0.65464675,  0.61473066]], dtype=float32)

In [60]:
######################################################################################################################
#################################################DEBUG################################################################
######################################################################################################################
X = train_features.values            
y = train_labels['AU'].values        
clf1 = xgb.train(params=params, dtrain=xgb.DMatrix(X, y), num_boost_round=10)                    
foo =clf1.predict(xgb.DMatrix(test_features.values))    
a = np.asarray(foo)
np.savetxt("C:\\git\\Airbnb\\fooarray.csv", a, delimiter=",")

pred_listF= []
model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,       
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


pred_listF = []
for col in targetCV.columns:
    if col == 'AU': 
        model2.fit(train_features, train_labels[col])
        pred = model2.predict_proba(test_features) 
        pred_listF.append([col, pred])
        
dfResult2F = pd.DataFrame()                
dfResult2F['id'] = ids# can use the same ids created on the previous step

for a in pred_listF: # gets the predictions, one country at a time
    dest = a[0] #get country name"
    predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
    values = []
    for p in predictions:
        values.append(p[1])
    dfResult2F[dest] = values                

#list_of_labels2F = get_top5_on_lis_of_labels2F(dfResult2F, False)               

In [64]:
pred_listF

[['AU', array([[  9.98124897e-01,   1.87509728e-03],
         [  9.94539440e-01,   5.46054821e-03],
         [  9.98789132e-01,   1.21086813e-03],
         ..., 
         [  9.98976052e-01,   1.02392782e-03],
         [  9.98894095e-01,   1.10592216e-03],
         [  9.99362469e-01,   6.37520803e-04]], dtype=float32)]]

In [63]:
model2.classes_

[False, True]

In [26]:
ndfTH = 0.2
def get_top5_on_lis_of_labels2(dfResult, submission):
    list_of_labels = []    

    for _, row in dfResult.iterrows():
        l = []
        done = 0
        pred_country = 'error'
        i = 5
        
        
        if max(row[1:11]) <= 0.35:
            i = 4
            if submission:
                list_of_labels.append([row['id'], 'NDF'])
            else:
                l.append('NDF') 

        
#         if row['AU'] < ndfTH and row['CA'] < ndfTH and row['DE'] < ndfTH and row['ES'] <ndfTH and row['FR'] < ndfTH and row['GB'] < ndfTH and row ['IT'] < ndfTH and row['NL'] < ndfTH and row['PT'] < ndfTH:
#             if row['US'] < 0.5:
#                 if row['other'] < 0.3:
#                     pred_country = 'NDF'
#                     done = 1
        
        
#         if row['FR'] > 0.20:
#             i = 4
#             if submission:
#                 list_of_labels.append([row['id'], 'FR'])
#             else:
#                 l.append('FR') 
            
        for option in (
                        map (
                            it(0,1), row[1:].order(ascending=0)[:i].iteritems()
                            )
                      ):
            country = option[0]
            prob = option[1]
            if done == 0:
                pred_country = country
                #if prob > 0.9 and country == 'US':
                #    pred_country = country
                #    done = 1                    
                #if prob >= 0.8: 
                #    pred_country = country
                #    done = 1
                #if prob <= 0.3:                 
                #    pred_country = 'NDF'
                #    done = 1
            if submission:
                list_of_labels.append([row['id'], pred_country]) #for submissions
            else:
                l.append(pred_country)   #for train CV
        
        if submission == False:
            list_of_labels.append(l)
            
    return list_of_labels  

In [43]:
# #simple tree for graph:
# import pydot 
# from sklearn import tree
# from sklearn.externals.six import StringIO  
# features = ['PopInKtoAU','PopInKtoES','PopInKtoDE', 'at_checkpoint']


# #train.columns
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(train[features], target)
# tree.export_graphviz(clf,  out_file='C:\\git\\Airbnb\\t2.dot')


# dot_data = StringIO() 
# tree.export_graphviz(clf, out_file=dot_data) 
# graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
# graph.write_pdf("C:\\git\\Airbnb\\t2.pdf") 

True

In [27]:
import operator
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()

create_feature_map(list(train.ix[:, train.columns != 'country_destination'].columns.values))
importance = clf1.get_fscore(fmap='xgb.fmap')
importance_df = pd.DataFrame(
    sorted(importance.items(), key=operator.itemgetter(1)), 
    columns=['feature','fscore']
    )    
#importance_df.to_csv('C:\\git\\Airbnb\\importance_df.csv',index=False) 

In [45]:
#model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
#            metric_params=None, n_neighbors=4, p=2, weights='uniform')

# model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#               max_depth=7, max_features=9, max_leaf_nodes=None,
#               min_samples_leaf=1, min_samples_split=2,
#               min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
#               oob_score=False, random_state=None, verbose=0,
#               warm_start=False)
#model = RandomForestClassifier(n_estimators=50,  criterion='gini')
#Best
#model = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
#                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0) 

#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=30,
#                    objective='multi:softprob', subsample=0.7, colsample_bytree=0.6, seed=0)     

model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)     

xgbparams = {'eta': 0.1,
          'max_depth': 6,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'objective': 'binary:logistic',
          'num_class': 2} 

In [46]:
#RUN CROSS VALIDATION
mycolumns = ['feature_list', 'scoreM1', 'ScoreM2NoNDF', 'ScoreM2Full', 'ScoreM3']
resultdf2 = pd.DataFrame(columns=mycolumns)


#features = train.columns# train.ix[:, train.columns != 'secs_elapsed'].columns #  #k_best_features.keys()
features = train [ [col for col in train.columns if col not in ['start_from_zero', 
                                                                'isstart_from_zero_0', 'isstart_from_zero_1',
                                                                'num_actions', ]] ].columns
 
#features = k_best_features.keys()
#features.append('country_destination')


score1, ScoreM2NoNDF, ScoreM2Full, Score3 = crossValidate(model1, model2, xgbparams, train[features], dfpivottarget, 
                                                          nfolds = 3, 
                                                          debug= True,
                                                          runM1=False, 
                                                          runM2=False, 
                                                          runM2F=True, 
                                                          runM3=True)

resultdf2.loc[len(resultdf2)] = (features, score1, ScoreM2NoNDF, ScoreM2Full, Score3)
resultdf2

Cross Validating...
Go for fold:  0
fold: 0 score: 0 | 0 | 0.854961552739 | 0.819028063439
Go for fold:  1
fold: 1 score: 0 | 0 | 0.853833848079 | 0.819105522224
Go for fold:  2
fold: 2 score: 0 | 0 | 0.854957023789 | 0.818516099969


,feature_list,scoreM1,ScoreM2NoNDF,ScoreM2Full,ScoreM3
0,"Index([u'about_us', u'account', u'acculynk_bin...",NaN,NaN,0.854584,0.818883


In [ ]:
#START ACTUAL PREDCTION

In [25]:
# model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#              max_depth=7, max_features=9, max_leaf_nodes=None,
#              min_samples_leaf=1, min_samples_split=2,
#              min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
#              oob_score=False, random_state=None, verbose=0,
#              warm_start=False)

# model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
#             metric_params=None, n_neighbors=4, p=2, weights='uniform')

#model = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
#                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0) 

#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=43,
#                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6, seed=0) 

model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)    

In [71]:
############################### EVAL  MODEL1
#features = train.columns#  k_best_features.keys() #


model1.fit(train[features], target)
pred_array = model1.predict_proba(test[features])


list_of_labels1 = []
print "Start" + time.strftime("%c")
for idx, val in enumerate(pred_array):
    mypred = (zip(model1.classes_, pred_array[idx]))
    sorted_by_pred = sorted(mypred, key=lambda tup: tup[1], reverse=True)
    sorted_by_pred = sorted_by_pred[:5]
    only_labels =  [x[0] for x in sorted_by_pred]
    
    #unpivot for submission:
    for country in only_labels:   
        list_of_labels1.append([test_ids[idx], country] )
print "End1 " + time.strftime("%c")        
        
preds_df = pd.DataFrame(list_of_labels1, columns=('id', 'country'))       
print "End2 " + time.strftime("%c")        

Start01/07/16 18:32:53
End1 01/07/16 18:32:56
End2 01/07/16 18:32:56


In [72]:
######EXPORT MODEL 1
preds_df.to_csv('C:\\git\\Airbnb\\m1.csv',index=False) 

In [26]:
#######MODEL2
#features = train.columns#  k_best_features.keys() #
features = train.ix[:, train.columns != 'country_destination'].columns  #k_best_features.keys()
Model2_list = []

for col in dfpivottarget.columns:
    if col != 'country_destination':
        print col
        model2.fit(train[features], dfpivottarget[col])
        pred = model2.predict_proba(test[features])  

        Model2_list.append([col, pred])

AU
CA
DE
ES
FR
GB
IT
NDF
NL
other
PT
US


In [27]:
#Model2_list.classes_ # check it is array([False,  True], dtype=bool)
dfResult = pd.DataFrame()
dfResult['id'] = test_ids

for a in Model2_list:
    dest = a[0] 
    predictions = a[1]
    values = []
    for p in predictions:
        values.append(p[1])
    dfResult[dest] = values

In [28]:
list_of_labels2 = get_top5_on_lis_of_labels2F(dfResult, True)

In [29]:
preds_df = pd.DataFrame(list_of_labels2, columns=('id', 'country'))
preds_df.to_csv('C:\\git\\Airbnb\\foo.csv',index=False) 

In [43]:
foo =dfResult[dfResult['id']=='j5kxtvyc9b']
foo

,id,AU,CA,DE,ES,FR,GB,IT,NL,other,PT,US
0,j5kxtvyc9b,0.00064,0.000915,0.001786,0.007188,0.018538,0.009074,0.013322,0.001011,0.022586,0.001541,0.077482


In [50]:
get_top5_on_lis_of_labels2(foo, True)

[['j5kxtvyc9b', 'US'],
 ['j5kxtvyc9b', 'other'],
 ['j5kxtvyc9b', 'FR'],
 ['j5kxtvyc9b', 'IT'],
 ['j5kxtvyc9b', 'GB']]

In [61]:
it(1,2)('ABCDEFG')

('B', 'C')

In [97]:
string = 'aabbcdd'

In [98]:
len(string)

7